In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time
from pyspark.sql.functions import avg, col, year, to_date

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/16 17:42:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/16 17:42:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/07/16 17:42:12 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Show DataFrame
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [8]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [9]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
# Extract the year from the date column
df_year = df.withColumn('year', year(to_date(col('date'), 'yyyy-MM-dd')))

# Calculate avg price, and group by bedrooms and year
averages = df_year.groupBy('year','bedrooms').agg(avg('price').alias('average_price'))

# Filter for 4-bedroom houses only
four_bdrm_averages = averages.filter(averages.bedrooms == 4)

# Select year and avg price columns, sort by year
result = four_bdrm_averages.orderBy('year').select('year', 'average_price')

# Show results rounded to two decimals
result.withColumn('average_price', result['average_price'].cast('decimal(10,2)')).show()

print("The average price of a four bedroom house sold per year is shown above.")

+----+-------------+
|year|average_price|
+----+-------------+
|2019|    300263.70|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+

The average price of a four bedroom house sold per year is shown above.


In [10]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

# Calculate avg price, and group by bedrooms and year
averages = df.groupBy('date_built','bedrooms','bathrooms').agg(avg('price').alias('average_price'))

# Filter for 3-bedroom, 3-bath houses only
three_bdrm_averages = averages.filter((averages.bedrooms == 3) & (averages.bathrooms == 3))

# Select year and avg price columns, sort by year
result = three_bdrm_averages.orderBy('date_built').select('date_built', 'average_price')

# Show results rounded to two decimals
result.withColumn('average_price', result['average_price'].cast('decimal(10,2)')).show()

print("The average price of a 3-bedroom, 3-bathroom house sold by year built is shown above.")

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|    288770.30|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+

The average price of a 3-bedroom, 3-bathroom house sold by year built is shown above.


In [7]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

# First filter for 3-bedroom, 3-bath houses only
filtered_df = df.filter((df.bedrooms == 3) & (df.bathrooms == 3) & (df.floors == 2)
                                     & (df.sqft_living >= 2000))
# Calculate avg price, and group by bedrooms and year
averages = filtered_df.groupBy('date_built').agg(avg('price').alias('average_price'))

# Select year and avg price columns, sort by year
result = averages.orderBy('date_built').select('date_built', 'average_price')

# Show results rounded to two decimals
result.withColumn('average_price', result['average_price'].cast('decimal(10,2)')).show()

print("The average price of a 3-bedroom, 3-bathroom house sold by year built with two floors, greater than or equal to 2,000 square feet is shown above.")


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|    293965.10|
|      2017|    280317.58|
+----------+-------------+

The average price of a 3-bedroom, 3-bathroom house sold by year built with two floors, greater than or equal to 2,000 square feet is shown above.


In [13]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.
start_time = time.time()

# Calculate avg price, and group by bedrooms and year
averages = df.groupBy('view').agg(avg('price').alias('average_price'))

# Filter for 3-bedroom, 3-bath houses only
filtered_averages = averages.filter(col('average_price') >= 350000)

# Select year and avg price columns, sort by year
result = filtered_averages.orderBy(col('view').desc()).select('view','average_price')

# Show results rounded to two decimals
result.withColumn('average_price', result['average_price'].cast('decimal(10,2)')).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|   1026669.50|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|   1054325.60|
|  94|   1033536.20|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|   1072285.20|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|   1063498.00|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 1.9175407886505127 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.


In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 


In [ ]:
# 11. Read the formatted parquet data.


In [ ]:
# 12. Create a temporary table for the parquet data.


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.


In [ ]:
# 15. Check if the home_sales is no longer cached

